In [ ]:
import pandas as pd
import numpy as np
from math import ceil

In [ ]:
# first of all, we import our data
data=pd.read_excel('rfm.xlsx')[['customer_id','date_created_gmt','total']]
# since the order_id won't be useful, we set the 'customer_id' as our index
data=data.set_index('customer_id').sort_index()
# we modify the column 'date_created_gmt' using REGEX : there is a letter T in the middle of the date string
# which doesn't allow us to convert it into a date type
data['date_created_gmt']=data['date_created_gmt'].replace(to_replace='[T]', value=' ',regex=True).transform(pd.Timestamp)
# we add a 'today' column which we will use later on to determine the recency
data['today']=pd.Timestamp.today()

# Approach :
The idea is to divide the data into different tables: 'recency_table', 'frequency_table' and 'monetary_table'.Then  determine the score and finally merge all tables to get the data that we want

In [ ]:
def score_recency (group):
    duree=np.min(group['today'].dt.to_period('D').view(dtype='int64')-group['date_created_gmt'].dt.to_period('D').view(dtype='int64'))
    if duree<5:
        return 5
    elif 5<=duree<8:return 4
    elif 8<=duree<21:return 3
    elif 21<=duree<=33:return 2
    return 1
def frequency(group):
    freq= np.sum(1*(group['total']>0))

    return freq
def monetary(group):
    return np.sum(group['total'])

def frequency_score(): # this function divides the 'frequency' table into segments of 20%s and return a column score
    # number_of_lines -> 100%
    # x               -> 20%
    n=len(frequency_table)
    d=frequency_table['frequency'].copy().to_dict()
    l=[]
    for x in d:
        l.append(d[x])

    l2={}
    l=list(set(l))
    l.sort(reverse=True)
    score =5
    mn=1
    n=len(l)
    for i in range(n):
        if i+1==int(mn*n/5):
            mn+=1
            score-=1
        l2[l[i]]=max(1,score)
    for x in d:
        d[x]=l2[d[x]]
    frequency_table['frequency_score']=pd.Series(d).copy().to_frame()
    return frequency_table
def monetary_score(): # this function divides the table into segments of 20%s and return a column score
    # number_of_lines -> 100%
    # x               -> 20%
    n=len(monetary_table)
    d=monetary_table['monetary'].copy().to_dict()
    l=[]
    for x in d:
        l.append(d[x])
    l.sort(reverse=True)
    l2={}

    score =5
    mn=1
    for i in range(n):
        if i+1==int(mn*n/5):
            mn+=1
            score-=1
        if l[i] not in l2:
            l2[l[i]]=score
    for x in d:
        d[x]=l2[d[x]]
    monetary_table['monetary_score']=pd.Series(d).copy().to_frame()
    return monetary_table

In [ ]:
recency_table=data.groupby('customer_id').apply(score_recency).to_frame('recency_score').dropna()
frequency_table=data.groupby('customer_id').apply(frequency).to_frame('frequency').dropna()
monetary_table=data.groupby('customer_id').apply(monetary).to_frame('monetary').dropna()

In [ ]:
frequency_table=frequency_score()
monetary_table=monetary_score()

In [ ]:
data=pd.merge(pd.merge(recency_table,frequency_table,how='inner',left_index=True, right_index=True),monetary_table,how='inner',left_index=True, right_index=True)

In [ ]:
data['RFM_score']=data.apply(lambda x:(x['frequency_score']+x['monetary_score']+x['recency_score'])/3,axis=1)
data=data.sort_values(by='RFM_score',ascending=False)
data['tuple']=data.groupby('customer_id').apply(lambda x:(list(x['frequency_score'])+list(x['monetary_score'])+list(x['recency_score'])))

In [ ]:
data

,recency_score,frequency,frequency_score,monetary,monetary_score,RFM_score,tuple
customer_id,,,,,,,
1,2,30,5,41744.0,5,4.000000,"[5, 5, 2]"
38,3,11,4,7498.0,5,4.000000,"[4, 5, 3]"
521,3,10,4,3150.5,5,4.000000,"[4, 5, 3]"
515,2,10,4,3886.0,5,3.666667,"[4, 5, 2]"
67,2,10,4,2754.5,5,3.666667,"[4, 5, 2]"
...,...,...,...,...,...,...,...
3256,1,1,1,200.0,1,1.000000,"[1, 1, 1]"
2249,1,1,1,140.0,1,1.000000,"[1, 1, 1]"
3258,1,1,1,182.0,1,1.000000,"[1, 1, 1]"


Now we can visualize the data and determine the types of our clients